In [1]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import torch

In [2]:
from scipy.stats import uniform, norm
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [8]:
def sigma_to_rho(sigma):
    rho = torch.log(torch.exp(sigma) - 1)
    return rho

In [9]:
def rho_to_sigma(rho):
    sigma = torch.log(1 + torch.exp(rho))
    return sigma

In [10]:
def log_q_pdf(w, theta):
    (mu_A, mu_b, rho_A, rho_b) = theta 
    
    sigma_A = rho_to_sigma(rho_A)
    qA = torch.distributions.normal.Normal(mu_A, sigma_A)
    
    sigma_b = rho_to_sigma(rho_b)
    qb = torch.distributions.normal.Normal(mu_b, sigma_b)
    
    lp = qA.log_prob(w[0]) + qb.log_prob(w[1])
    return lp

In [11]:
def log_p_pdf(w):
    prior_mu_A = torch.tensor(0.0) 
    prior_sigma_A = torch.tensor(1.0)
    pA = torch.distributions.normal.Normal(prior_mu_A, prior_sigma_A)
    
    prior_mu_b = torch.tensor(0.0) 
    prior_sigma_b = torch.tensor(1.0)
    pb = torch.distributions.normal.Normal(prior_mu_b, prior_sigma_b)
    
    lp = pA.log_prob(w[0]) + pb.log_prob(w[1])
    return lp

In [131]:
def f(w, theta):
    v = log_q_pdf(w, theta) - log_p_pdf(w)
    return v

In [82]:
def approximate_KL(theta):
    prior_mu_A = torch.tensor(0.0) 
    prior_sigma_A = torch.tensor(1.0)
    pA = torch.distributions.normal.Normal(prior_mu_A, prior_sigma_A)
    prior_mu_b = torch.tensor(0.0) 
    prior_sigma_b = torch.tensor(1.0)
    pb = torch.distributions.normal.Normal(prior_mu_b, prior_sigma_b)    
    
    (mu_A, mu_b, rho_A, rho_b) = theta
    
    sigma_A = rho_to_sigma(rho_A)
    qA = torch.distributions.normal.Normal(mu_A, sigma_A)
    sigma_b = rho_to_sigma(rho_b)
    qb = torch.distributions.normal.Normal(mu_b, sigma_b)
    
    lst_lp2 = []
    for j in range(0,100000):
        A = qA.sample()
        b = qb.sample()

        lp = 0
        lp += qA.log_prob(A) + qb.log_prob(b)
        lp -= pA.log_prob(A) + pb.log_prob(b)
        

        lst_lp2.append(lp)
    return torch.stack(lst_lp2).mean()

In [84]:
def prior_KL(theta):
    prior_mu_A = torch.tensor(0.0) 
    prior_sigma_A = torch.tensor(1.0)
    pA = torch.distributions.normal.Normal(prior_mu_A, prior_sigma_A)
    prior_mu_b = torch.tensor(0.0) 
    prior_sigma_b = torch.tensor(1.0)
    pb = torch.distributions.normal.Normal(prior_mu_b, prior_sigma_b)    
    
    (mu_A, mu_b, rho_A, rho_b) = theta
    
    sigma_A = rho_to_sigma(rho_A)
    qA = torch.distributions.normal.Normal(mu_A, sigma_A)
    sigma_b = rho_to_sigma(rho_b)
    qb = torch.distributions.normal.Normal(mu_b, sigma_b)
    return torch.distributions.kl_divergence(qA, pA) + torch.distributions.kl_divergence(qb, pb)

In [154]:
prior_mu_A = torch.tensor(0.0) 
prior_sigma_A = torch.tensor(1.0)
pA = torch.distributions.normal.Normal(prior_mu_A, prior_sigma_A)
prior_mu_b = torch.tensor(0.0) 
prior_sigma_b = torch.tensor(1.0)
pb = torch.distributions.normal.Normal(prior_mu_b, prior_sigma_b)   

In [155]:
mu_A = torch.tensor(1.0)
sigma_A = torch.tensor(0.1)
rho_A = sigma_to_rho(sigma_A)
mu_b = torch.tensor(-0.4)
sigma_b = torch.tensor(0.1)
rho_b = sigma_to_rho(sigma_b)

theta = torch.tensor([mu_A, mu_b, rho_A, rho_b], requires_grad=True)

In [156]:
KL = prior_KL(theta)

In [157]:
KL.backward()

In [158]:
theta.grad

tensor([ 1.0000, -0.4000, -0.9421, -0.9421])

In [159]:

lst_grad_mu = []
lst_grad_rho = []
for j in range(0,5000):
    epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

    with torch.no_grad():
        w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
    w.requires_grad = True

    v = f(w, theta)   
    v.backward()
    v = torch.tensor([])

    with torch.no_grad():
        grad_mu = w.grad + theta.grad[0:2]
        grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


    lst_grad_mu.append(grad_mu)
    lst_grad_rho.append(grad_rho)

    theta.grad.zero_()
    w.grad.zero_()
    v.detach()
grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
        
 



In [160]:
grad_mu

tensor([ 0.9999, -0.3989])

In [161]:
grad_rho

tensor([-1.0426, -0.9716])

In [ ]:
alpha = torch.tensor(10e-5)

In [169]:
for i in range(0,10000):
    lst_grad_mu = []
    lst_grad_rho = []
    for j in range(0,10):
        epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

        with torch.no_grad():
            w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
        w.requires_grad = True

        v = f(w, theta)   
        v.backward()
        v = torch.tensor([])

        with torch.no_grad():
            grad_mu = w.grad + theta.grad[0:2]
            grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


        lst_grad_mu.append(grad_mu)
        lst_grad_rho.append(grad_rho)

        theta.grad.zero_()
        w.grad.zero_()
        v.detach()
    grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
    grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
    
    #print(grad_mu)
    #print(grad_rho)
        
    with torch.no_grad():
        theta[0:2] -= alpha*grad_mu
        theta[2:4] -= alpha*grad_rho
    
    print(theta)



tensor([ 0.1563, -0.0649, -0.6995, -0.6748], requires_grad=True)
tensor([ 0.1559, -0.0649, -0.6988, -0.6735], requires_grad=True)
tensor([ 0.1557, -0.0650, -0.6973, -0.6724], requires_grad=True)
tensor([ 0.1557, -0.0650, -0.6972, -0.6723], requires_grad=True)
tensor([ 0.1553, -0.0650, -0.6972, -0.6716], requires_grad=True)
tensor([ 0.1551, -0.0651, -0.6973, -0.6709], requires_grad=True)
tensor([ 0.1552, -0.0652, -0.6963, -0.6691], requires_grad=True)
tensor([ 0.1549, -0.0652, -0.6955, -0.6689], requires_grad=True)
tensor([ 0.1546, -0.0651, -0.6939, -0.6681], requires_grad=True)
tensor([ 0.1544, -0.0649, -0.6934, -0.6680], requires_grad=True)
tensor([ 0.1543, -0.0650, -0.6930, -0.6677], requires_grad=True)
tensor([ 0.1542, -0.0651, -0.6929, -0.6676], requires_grad=True)
tensor([ 0.1540, -0.0652, -0.6933, -0.6676], requires_grad=True)
tensor([ 0.1538, -0.0650, -0.6932, -0.6666], requires_grad=True)
tensor([ 0.1536, -0.0648, -0.6933, -0.6646], requires_grad=True)
tensor([ 0.1534, -0.0646,

tensor([ 0.1362, -0.0559, -0.6088, -0.5811], requires_grad=True)
tensor([ 0.1361, -0.0560, -0.6073, -0.5810], requires_grad=True)
tensor([ 0.1357, -0.0560, -0.6068, -0.5807], requires_grad=True)
tensor([ 0.1355, -0.0559, -0.6070, -0.5803], requires_grad=True)
tensor([ 0.1354, -0.0558, -0.6074, -0.5795], requires_grad=True)
tensor([ 0.1353, -0.0556, -0.6074, -0.5792], requires_grad=True)
tensor([ 0.1350, -0.0556, -0.6068, -0.5785], requires_grad=True)
tensor([ 0.1349, -0.0558, -0.6063, -0.5778], requires_grad=True)
tensor([ 0.1346, -0.0556, -0.6059, -0.5762], requires_grad=True)
tensor([ 0.1346, -0.0556, -0.6052, -0.5766], requires_grad=True)
tensor([ 0.1344, -0.0556, -0.6045, -0.5755], requires_grad=True)
tensor([ 0.1343, -0.0555, -0.6047, -0.5754], requires_grad=True)
tensor([ 0.1342, -0.0555, -0.6048, -0.5749], requires_grad=True)
tensor([ 0.1341, -0.0554, -0.6046, -0.5737], requires_grad=True)
tensor([ 0.1343, -0.0555, -0.6032, -0.5733], requires_grad=True)
tensor([ 0.1341, -0.0555,

tensor([ 0.1187, -0.0491, -0.5323, -0.4860], requires_grad=True)
tensor([ 0.1186, -0.0488, -0.5320, -0.4853], requires_grad=True)
tensor([ 0.1182, -0.0490, -0.5300, -0.4842], requires_grad=True)
tensor([ 0.1181, -0.0487, -0.5303, -0.4834], requires_grad=True)
tensor([ 0.1181, -0.0487, -0.5304, -0.4819], requires_grad=True)
tensor([ 0.1179, -0.0486, -0.5287, -0.4816], requires_grad=True)
tensor([ 0.1178, -0.0486, -0.5278, -0.4811], requires_grad=True)
tensor([ 0.1176, -0.0486, -0.5280, -0.4812], requires_grad=True)
tensor([ 0.1175, -0.0483, -0.5263, -0.4807], requires_grad=True)
tensor([ 0.1174, -0.0484, -0.5256, -0.4800], requires_grad=True)
tensor([ 0.1173, -0.0487, -0.5244, -0.4791], requires_grad=True)
tensor([ 0.1169, -0.0486, -0.5238, -0.4784], requires_grad=True)
tensor([ 0.1166, -0.0484, -0.5242, -0.4781], requires_grad=True)
tensor([ 0.1167, -0.0485, -0.5242, -0.4773], requires_grad=True)
tensor([ 0.1164, -0.0483, -0.5236, -0.4768], requires_grad=True)
tensor([ 0.1164, -0.0483,

tensor([ 0.1035, -0.0422, -0.4462, -0.4136], requires_grad=True)
tensor([ 0.1032, -0.0422, -0.4457, -0.4113], requires_grad=True)
tensor([ 0.1031, -0.0424, -0.4450, -0.4101], requires_grad=True)
tensor([ 0.1029, -0.0425, -0.4444, -0.4102], requires_grad=True)
tensor([ 0.1029, -0.0427, -0.4446, -0.4092], requires_grad=True)
tensor([ 0.1030, -0.0429, -0.4440, -0.4075], requires_grad=True)
tensor([ 0.1026, -0.0426, -0.4427, -0.4067], requires_grad=True)
tensor([ 0.1022, -0.0423, -0.4422, -0.4057], requires_grad=True)
tensor([ 0.1021, -0.0420, -0.4413, -0.4052], requires_grad=True)
tensor([ 0.1019, -0.0421, -0.4397, -0.4045], requires_grad=True)
tensor([ 0.1019, -0.0421, -0.4389, -0.4031], requires_grad=True)
tensor([ 0.1018, -0.0421, -0.4388, -0.4021], requires_grad=True)
tensor([ 0.1016, -0.0420, -0.4388, -0.4020], requires_grad=True)
tensor([ 0.1013, -0.0418, -0.4381, -0.4013], requires_grad=True)
tensor([ 0.1011, -0.0415, -0.4372, -0.4007], requires_grad=True)
tensor([ 0.1009, -0.0414,

tensor([ 0.0887, -0.0364, -0.3634, -0.3269], requires_grad=True)
tensor([ 0.0886, -0.0364, -0.3630, -0.3262], requires_grad=True)
tensor([ 0.0886, -0.0364, -0.3624, -0.3259], requires_grad=True)
tensor([ 0.0884, -0.0364, -0.3623, -0.3256], requires_grad=True)
tensor([ 0.0883, -0.0366, -0.3619, -0.3255], requires_grad=True)
tensor([ 0.0882, -0.0369, -0.3620, -0.3248], requires_grad=True)
tensor([ 0.0883, -0.0371, -0.3620, -0.3246], requires_grad=True)
tensor([ 0.0879, -0.0368, -0.3611, -0.3239], requires_grad=True)
tensor([ 0.0876, -0.0364, -0.3610, -0.3233], requires_grad=True)
tensor([ 0.0874, -0.0363, -0.3607, -0.3227], requires_grad=True)
tensor([ 0.0873, -0.0362, -0.3610, -0.3228], requires_grad=True)
tensor([ 0.0874, -0.0360, -0.3607, -0.3220], requires_grad=True)
tensor([ 0.0876, -0.0358, -0.3593, -0.3212], requires_grad=True)
tensor([ 0.0875, -0.0358, -0.3592, -0.3197], requires_grad=True)
tensor([ 0.0876, -0.0360, -0.3591, -0.3196], requires_grad=True)
tensor([ 0.0875, -0.0360,

...

tensor([ 0.0078, -0.0046,  0.5509,  0.5319], requires_grad=True)
tensor([ 0.0078, -0.0052,  0.5506,  0.5321], requires_grad=True)
tensor([ 0.0078, -0.0055,  0.5505,  0.5321], requires_grad=True)
tensor([ 0.0071, -0.0051,  0.5511,  0.5320], requires_grad=True)
tensor([ 0.0075, -0.0050,  0.5512,  0.5317], requires_grad=True)
tensor([ 0.0073, -0.0052,  0.5510,  0.5317], requires_grad=True)
tensor([ 0.0076, -0.0052,  0.5511,  0.5315], requires_grad=True)
tensor([ 0.0079, -0.0051,  0.5507,  0.5319], requires_grad=True)
tensor([ 0.0083, -0.0049,  0.5508,  0.5317], requires_grad=True)
tensor([ 0.0083, -0.0045,  0.5508,  0.5316], requires_grad=True)
tensor([ 0.0082, -0.0046,  0.5506,  0.5315], requires_grad=True)
tensor([ 0.0082, -0.0047,  0.5502,  0.5314], requires_grad=True)
tensor([ 0.0075, -0.0043,  0.5508,  0.5317], requires_grad=True)
tensor([ 0.0081, -0.0045,  0.5509,  0.5315], requires_grad=True)
tensor([ 0.0081, -0.0044,  0.5507,  0.5314], requires_grad=True)
tensor([ 0.0082, -0.0045,

tensor([ 0.0116, -0.0056,  0.5545,  0.5331], requires_grad=True)
tensor([ 0.0112, -0.0060,  0.5544,  0.5336], requires_grad=True)
tensor([ 0.0111, -0.0061,  0.5539,  0.5337], requires_grad=True)
tensor([ 0.0110, -0.0056,  0.5541,  0.5337], requires_grad=True)
tensor([ 0.0112, -0.0055,  0.5541,  0.5336], requires_grad=True)
tensor([ 0.0110, -0.0054,  0.5540,  0.5334], requires_grad=True)
tensor([ 0.0121, -0.0054,  0.5544,  0.5333], requires_grad=True)
tensor([ 0.0120, -0.0056,  0.5541,  0.5331], requires_grad=True)
tensor([ 0.0120, -0.0059,  0.5538,  0.5333], requires_grad=True)
tensor([ 0.0122, -0.0063,  0.5535,  0.5340], requires_grad=True)
tensor([ 0.0122, -0.0060,  0.5531,  0.5339], requires_grad=True)
tensor([ 0.0122, -0.0054,  0.5528,  0.5338], requires_grad=True)
tensor([ 0.0122, -0.0061,  0.5527,  0.5341], requires_grad=True)
tensor([ 0.0122, -0.0063,  0.5533,  0.5340], requires_grad=True)
tensor([ 0.0121, -0.0065,  0.5533,  0.5338], requires_grad=True)
tensor([ 0.0124, -0.0066,

tensor([ 0.0126, -0.0068,  0.5496,  0.5365], requires_grad=True)
tensor([ 0.0126, -0.0071,  0.5491,  0.5364], requires_grad=True)
tensor([ 0.0122, -0.0066,  0.5491,  0.5368], requires_grad=True)
tensor([ 0.0125, -0.0069,  0.5491,  0.5369], requires_grad=True)
tensor([ 0.0120, -0.0071,  0.5490,  0.5371], requires_grad=True)
tensor([ 0.0122, -0.0073,  0.5492,  0.5371], requires_grad=True)
tensor([ 0.0122, -0.0069,  0.5491,  0.5369], requires_grad=True)
tensor([ 0.0121, -0.0070,  0.5486,  0.5370], requires_grad=True)
tensor([ 0.0115, -0.0065,  0.5489,  0.5367], requires_grad=True)
tensor([ 0.0115, -0.0064,  0.5487,  0.5372], requires_grad=True)
tensor([ 0.0121, -0.0060,  0.5485,  0.5370], requires_grad=True)
tensor([ 0.0110, -0.0056,  0.5495,  0.5368], requires_grad=True)
tensor([ 0.0113, -0.0062,  0.5500,  0.5372], requires_grad=True)
tensor([ 0.0117, -0.0061,  0.5501,  0.5368], requires_grad=True)
tensor([ 0.0116, -0.0060,  0.5497,  0.5364], requires_grad=True)
tensor([ 0.0120, -0.0058,

tensor([ 0.0085, -0.0052,  0.5515,  0.5396], requires_grad=True)
tensor([ 0.0082, -0.0052,  0.5518,  0.5395], requires_grad=True)
tensor([ 0.0085, -0.0049,  0.5518,  0.5391], requires_grad=True)
tensor([ 0.0085, -0.0049,  0.5515,  0.5394], requires_grad=True)
tensor([ 0.0087, -0.0042,  0.5516,  0.5397], requires_grad=True)
tensor([ 0.0084, -0.0044,  0.5516,  0.5399], requires_grad=True)
tensor([ 0.0081, -0.0043,  0.5517,  0.5398], requires_grad=True)
tensor([ 0.0080, -0.0041,  0.5515,  0.5397], requires_grad=True)
tensor([ 0.0085, -0.0042,  0.5516,  0.5396], requires_grad=True)
tensor([ 0.0083, -0.0036,  0.5517,  0.5400], requires_grad=True)
tensor([ 0.0088, -0.0037,  0.5520,  0.5398], requires_grad=True)
tensor([ 0.0091, -0.0033,  0.5517,  0.5396], requires_grad=True)
tensor([ 0.0088, -0.0035,  0.5516,  0.5394], requires_grad=True)
tensor([ 0.0090, -0.0033,  0.5515,  0.5394], requires_grad=True)
tensor([ 0.0088, -0.0031,  0.5514,  0.5390], requires_grad=True)
tensor([ 0.0088, -0.0026,

tensor([0.0072, 0.0016, 0.5494, 0.5442], requires_grad=True)
tensor([0.0075, 0.0016, 0.5493, 0.5439], requires_grad=True)
tensor([0.0072, 0.0018, 0.5492, 0.5439], requires_grad=True)
tensor([0.0072, 0.0022, 0.5489, 0.5436], requires_grad=True)
tensor([0.0074, 0.0024, 0.5489, 0.5434], requires_grad=True)
tensor([0.0078, 0.0027, 0.5487, 0.5436], requires_grad=True)
tensor([0.0081, 0.0026, 0.5487, 0.5439], requires_grad=True)
tensor([0.0079, 0.0025, 0.5485, 0.5443], requires_grad=True)
tensor([0.0076, 0.0028, 0.5484, 0.5446], requires_grad=True)
tensor([0.0067, 0.0023, 0.5485, 0.5446], requires_grad=True)
tensor([0.0070, 0.0026, 0.5494, 0.5445], requires_grad=True)
tensor([0.0074, 0.0024, 0.5490, 0.5444], requires_grad=True)
tensor([0.0072, 0.0024, 0.5489, 0.5441], requires_grad=True)
tensor([0.0074, 0.0028, 0.5486, 0.5440], requires_grad=True)
tensor([0.0069, 0.0027, 0.5490, 0.5439], requires_grad=True)
tensor([0.0071, 0.0025, 0.5488, 0.5442], requires_grad=True)
tensor([0.0073, 0.0025, 

In [170]:
rho_to_sigma(torch.tensor(0.5488))

tensor(1.0047)